# Functors in Scala

The functional programming style eschews loops and replaces it with tail recursive functions that express the logic in a simple fashion, without carrying extra state.  Another mechanism that we will study now is that of "functors" such as "map", "filter" and "fold". These functors allow us to manipulate Lists of objects. But they also apply to other data structures in scala such as Maps.


Before we begin with functors let us study different ways to write functions in scala, including a convenient notation for anonymous functions.

Let us start with a function to multiply every element of a list by two.

We will now multiply every element of a list by 2.

In [1]:
def multiplyEachEltByTwo(lst: List[Int], accList: List[Int] = Nil): List[Int] = lst match {
    case Nil => accList
    case hd::tail => {
        val newAccList = accList ++ List(2 * hd) // We add 2 * hd at the end why?
        multiplyEachEltByTwo(tail, newAccList)
    }
} 

defined function multiplyEachEltByTwo

Next, we would like to remove all the even elements from a list, returning a new list with just the odd elements.

In [2]:
def removeEvenNumbers(lst: List[Int], accList: List[Int] = Nil): List[Int] = lst match {
    case Nil => accList
    case hd::tail => {
        val newAccList = if (hd %2 == 0) { accList } else { accList ++ List(hd) }
        removeEvenNumbers(tail, newAccList)
    }
} 

defined function removeEvenNumbers

Finally, we wish to sum up the elements of the list

In [3]:
def sumOfList(lst: List[Int], sum: Int = 0): Int = lst match {
    case Nil => sum
    case hd::tail => sumOfList(tail, sum + hd)
}

defined function sumOfList

Finally, we can write our main function:

In [6]:
def processList(lst: List[Int]): Int = {
    // Multiply by two
    val lst1 = removeEvenNumbers(lst)
    val lst2 = multiplyEachEltByTwo(lst1)
    sumOfList(lst2)
}

defined function processList

In [9]:
removeEvenNumbers(List(1,3,4,5,6,7,8))
multiplyEachEltByTwo(List(1,2,3,4))
sumOfList ((1 to 100).toList)
processList((1 to 20).toList)

res8_0: List[Int] = List(1, 3, 5, 7)
res8_1: List[Int] = List(2, 4, 6, 8)
res8_2: Int = 5050
res8_3: Int = 200

This is somewhat painful since we need to write three separate functions to do the job.  Is there something better we can do?  Yes: let us recognize three patterns of operations we would like to achieve:

- Map: apply a function f to every element of a list.
- Filter: keep just those elements of the list that satisfy a "predicate"
- Fold (or reduce): perform an accumulative operation to every element of the list.

### Anonymous Functions In Scala 

Before we look closer at these operations, let us first familiarize ourselves with `anonymous` functions in scala.
Often it is cumbersome to define functions by name where we would like to pass a function. Therefore, we will use "anonymous" functions.

In [6]:
def multiplyByTwo(x: Int): Int = x * 2

defined function multiplyByTwo

Here are two other ways to write the same thing.

In [7]:
val f : Int => Int = (x) => x * 2

f: Int => Int = ammonite.$sess.cmd6$Helper$$Lambda$2668/1591065996@4874493

f is bound to a function that takes in an argument `x` and returns  `x * 2`. You can pass the expression ` (x) => x * 2 ` in any context you wish without giving it a name as we will see. Here is another succint version:

In [8]:
val f2: Int => Int = _ * 2

f2: Int => Int = ammonite.$sess.cmd7$Helper$$Lambda$2690/1232048499@361dc6d2

The `_` here is simply the first argument. Often it is important to specify the type of an argument in an anonymous function.

In [1]:
val g = (x: String) => x + x // OK: Scala infers the type of x + x from that of x and the type of g is inferred.

g: String => String = ammonite.$sess.cmd0$Helper$$Lambda$1999/0x00000008015a6840@5d35807

In [10]:
val g: String => String = x => x + x // OK: scala infers the typeof x from the type given to g

g: String => String = ammonite.$sess.cmd9$Helper$$Lambda$2712/496013557@578113b3

In [10]:
val g2 = x => x + x // BAD: Scala has no way of knowing what x is. It can be a String, Int, Double, ...

cmd10.sc:1: missing parameter type
val g2 = x => x + x // BAD: Scala has no way of knowing what x is. It can be a String, Int, Double, ...
         ^Compilation Failed

: 

Anonymous functions can take multiple arguments.

In [31]:
val addFun = (x: Int, y:Int) => x + y

addFun: (Int, Int) => Int = ammonite.$sess.cmd30$Helper$$Lambda$3086/901697560@464e089b

In [12]:
val addFun = (x: (Int, Int)) => x._1 + x._2

addFun: (Int, Int) => Int = ammonite.$sess.cmd11$Helper$$Lambda$2748/1806238802@472323aa

In [14]:
val addFun: (Int, Int) => Int = _ + _  // First _ is the first argument and second _ is the second argument.

addFun: (Int, Int) => Int = ammonite.$sess.cmd13$Helper$$Lambda$2766/1498942801@25bbfb2a

Last but not least in a case pattern matching setup, you can define an anonymous function without the match statement.

In [15]:
sealed trait MyList 
case object MyNil extends MyList
case class MyCons(x: Int, l: MyList) extends MyList

defined trait MyList
defined object MyNil
defined class MyCons

In [16]:
val anonIsEmptyFun: MyList => Boolean = (x) => { x match {
    case MyNil => true
    case MyCons(_, _) => false
}}

anonIsEmptyFun: MyList => Boolean = ammonite.$sess.cmd15$Helper$$Lambda$2896/492234261@675dcae8

In [17]:
val anonIsEmptyFun: MyList => Boolean = {
    case MyNil => true
    case MyCons(_, _) => false
}

anonIsEmptyFun: MyList => Boolean = ammonite.$sess.cmd16$Helper$$Lambda$2903/559238963@67bc868c

In other words, when you have the pattern 
~~~
 (x : Type) => x match {
     case .. =>
     case .. => 
     ...
     }
~~~
You can instead simply say 

~~~
{ 
  case .. => 
  case .. => 
  ...
 }
~~~

without saying `(x : Type) => x match`.

## Map, Filter and Fold (Reduce) Operations

In many languages, the use of for-loops/while loops to iterate is replaced by operations on data structures such as `map`, `filter` and `fold`. In this lecture, we provide a brief overview with some examples. We show how many varieties of loops or equivalently recursion, can be systematically replaced by these operations.


## Map operation

The idea of a map operation is to apply a function $f$ to every member of a container (eg., list, array, map, etc.) and return a new container.

### Example 1

We have a list `List(1, 3, 4, 5, 6, 110, 12, 2)`. We wish to compute the square of each element in the list and make a new list with the result.

In [18]:
def recursivelySquareEachElt(l: List[Int], acc: List[Int] = Nil): List[Int] = {
    if (l.length == 0)
        acc.reverse
    else
        recursivelySquareEachElt(l.tail, (l.head*l.head)::acc)
}

defined function recursivelySquareEachElt

In [19]:
recursivelySquareEachElt(List(10))

res18: List[Int] = List(100)

In [20]:
recursivelySquareEachElt(List(1, 3, 4, 5, 6, 110, 12, 2), Nil)

res19: List[Int] = List(1, 9, 16, 25, 36, 12100, 144, 4)

Using the map operator over lists.

In [21]:
def squareEachElt(l: List[Int]): List[Int] =  l.map( (x: Int) => x*x ) 
// x => x * x is an anonymous function that squares its arguments.

defined function squareEachElt

In [22]:
squareEachElt(List(1, 3, 4, 5, 6, 110, 12, 2))

res21: List[Int] = List(1, 9, 16, 25, 36, 12100, 144, 4)

`l.map(f)` says that apply the function `f` on each element of the list `f`.

First of all, the elements of the lists must be some type `A`, let's say. 
Next, the function `f` must be of type `A => B`.

Last but not least, `l.map(f)` applies `f` to every element in the list and returns a new list
of type `B`. Here is a recursive definition of this function. Can you make it tail recursive??

In [23]:
def listMap[A,B](lst: List[A], fun: A => B): List[B] = lst match {
    case Nil => Nil
    case hd :: tail => fun(hd) :: listMap(tail, fun)  // :: is the Cons operator in scala.
}

defined function listMap

In [24]:
def sayHelloTo(l: List[String]): List[String] = l.map( x => ("Hello "+ x)) // Type of x is inferred by Scala

defined function sayHelloTo

In [25]:
sayHelloTo(List("Cat", "Dog", "World"))

res24: List[String] = List("Hello Cat", "Hello Dog", "Hello World")

## Filter Operation.

Just like we have used map to apply a function to each element and make a new container, we use `filter` to remove all elements that do not satisfy a predicate.

__Predicate__ A preducate is a funciton that takes in a value and returns true/false.

`l.filter(c)` filters all those elements that do not satisfy the condition `c` from the list `l`.

In [26]:
def retainAllMultiplesOfThree(l: List[Int], acc: List[Int] = Nil): List[Int] = l match {
    case Nil => acc
    case hd :: tail => {
        val newAcc = if (hd % 3 == 0) { acc ++ List(hd)} else { acc }
        retainAllMultiplesOfThree(tail, newAcc)
    }
}

defined function retainAllMultiplesOfThree

In [27]:
def retainAllMultiplesOfThree(l: List[Int]): List[Int] = {
    l.filter( x => x%3 == 0 )
}

defined function retainAllMultiplesOfThree

In [11]:
retainAllMultiplesOfThree(List(10, 15, 18, 12, 3, 1, 5, 7, 8, 14))

res10: List[Int] = List(15, 18, 12, 3)

Here is how the filter operation is defined abstractly

In [28]:
def filterList[A] (lst: List[A], filterFun: A => Boolean): List[A] = lst match {
    case Nil => Nil
    case head :: tail => {
        if (filterFun(head)){
            head :: filterList(tail, filterFun)
        } else {
            filterList(tail, filterFun)
        }
    }
}

// Ths is not tail recursive. Why? Can you make it tail recursive?

defined function filterList

## Fold Operations

Fold/reduce operations are useful to gather all data thus far during a computation. Take a list

$$[l_1, l_2, \ldots, l_n] $$.

We wish to sum up the numbers in the list.
This is achieved in a loop with accumulator.
~~~
acc = 0
for each item in List
   acc = acc + item
return acc
~~~

We can also do it with fold left operator.

As an example consider the sum of the elements of a list above.



In [29]:
def recSumOfList(lst: List[Int], sum: Int = 0): Int = lst match {
    case Nil => sum
    case hd::tail => recSumOfList(tail, sum + hd)
}

defined function recSumOfList

Fold is a tricky operation to wrap one's head around. A list data structure gives us two versions of fold.

### list.foldLeft (startVal) (fun)

For list `[l1, l2, l3, ..., ln]` the function call computes the following unrolled function:

` fun(.... fun( fun ( fun( startVal, l1), l2), l3), ....., ln)`
This is equivalent to the following scala code:

~~~
var acc = startVal
for (lj <- list)
   acc =  fun(acc, lj) // Very imp: acc is the first argument and lj is the second argument.
~~~



### list.foldRight (startVal) (fun)

This iterates the list from right to left. To wit, list `[l1, l2, l3, ..., ln]` the function call computes the following unrolled function:

` fun(l1, fun(.....,fun(ln-2, fun(ln-1, fun(ln, startVal)))`

This is equivalent to the following scala code:

~~~
var acc = startVal
for (lj <- list.reverse) // Note list is iterated in reverse
   acc =  fun(lj, acc) // Very imp: acc is the second argument for foldRight
~~~

The fold function has two arguments: `startVal` and `fun`. Why don't we write: 
`list.foldLeft(startVal, fun)`? This is a special syntax for writing  functions with multiple argument 
in scala called  __curried syntax__

https://alvinalexander.com/scala/fp-book/partially-applied-functions-currying-in-scala

We will talk about currying in detail later on (in a few weeks) and it has nothing to do with Indian cuisine.
 

In [14]:
def sumList(l: List[Int]): Int = l.foldLeft (0) ((acc, x) => acc + x )
// Fold left with initial value of accumulator = 0
// Every time we have a new list element x and accumulator value acc, update acc by acc + x

defined function sumList

In [15]:
sumList(List(1, 2, 3,4, 5, 6, 7, 8, 9, 10))

res14: Int = 55

In [30]:
def sumFromRight(l: List[Int]) : Int = l.foldRight (0) ((x, acc) => x + acc)

defined function sumFromRight

Let us now write a function `reverseList`

In [34]:
def reverseList(l: List[Int]): List[Int] = 
l.foldLeft (Nil)  ( (listSoFar: List[Int], elt: Int) => {
    elt::listSoFar
} )

cmd34.sc:3: type mismatch;
 found   : List[Int]
 required: scala.collection.immutable.Nil.type
    elt::listSoFar
       ^Compilation Failed

: 

What just happened? Scala's type checker bailed on us.

- Nil is the empty list for any type: List[String], List[Int], List[Double], List[List[List[Int]]], and so on.
- The type checker is simply not sophisticated enough to figure out that the type of the accumulator in foldLeft here must be a list of int. 

There are two fixes.


In [35]:
def reverseListA(l: List[Int]): List[Int] = 
   l.foldLeft  ( List[Int]() ) ( (listSoFar: List[Int], elt: Int) => {
    elt::listSoFar
} )

defined function reverseListA

In [36]:
def reverseListB(l: List[Int]): List[Int] = 
   l.foldLeft[List[Int]]  ( Nil ) ( (listSoFar: List[Int], elt: Int) => {
    elt::listSoFar
} )

defined function reverseListB

In general, it is always nice to have the type of the accumulator specified in fold left. Last but not least, note that the anonymous function in fold can be written in case pattern form.

In [37]:
def reverseListC(l: List[Int]): List[Int] = l.foldLeft[List[Int]] (Nil) {
    case (listSoFar: List[Int], elt: Int) => elt::listSoFar
}

defined function reverseListC

In [38]:
reverseListA(List(1,2,3,4))

res37: List[Int] = List(4, 3, 2, 1)

In [39]:
reverseListB(List(1,2,3,4))

res38: List[Int] = List(4, 3, 2, 1)

In [40]:
reverseListC(List(1,2,3,4))

res39: List[Int] = List(4, 3, 2, 1)